In [80]:
import pandas as pd
import ast

In [101]:
businesses1 = pd.read_csv('yelp_results_from_missing_after_1_wb.csv', index_col=0)

In [102]:
businesses2 = pd.read_csv('yelp_results_from_missing_after_2_wb.csv', index_col=0)

In [103]:
businesses3 = pd.read_csv('yelp_results_from_now_closed_wb.csv', index_col=0)

In [108]:
businesses3.is_closed = True

In [174]:
sabina = pd.read_csv('CSVs/SH_all_business_query.csv', index_col=0)
jade = pd.read_csv('CSVs/all_Jade_data.csv', index_col=0)
maxk = pd.read_csv('CSVs/mk_complete.csv', index_col=0)
egemen = pd.read_csv('CSVs/egemen_all.csv', index_col=0)

In [192]:
all_businesses = pd.concat([businesses1, businesses2, businesses3,
                           sabina, jade, maxk, egemen]).drop_duplicates('id')

In [193]:
all_businesses.columns

Index(['id', 'alias', 'name', 'image_url', 'is_claimed', 'is_closed', 'url',
       'phone', 'display_phone', 'review_count', 'categories', 'rating',
       'location', 'coordinates', 'photos', 'hours', 'transactions',
       'business type', 'price', 'messaging', 'special_hours', 'error',
       'distance'],
      dtype='object')

In [194]:
all_businesses.drop(['phone', 'display_phone', 'coordinates', 'photos', 'hours', 'messaging', 'special_hours', 'error', 'distance', 'image_url', 'categories'],
                    axis=1, inplace=True)
all_businesses.rename(columns={'business type': 'business_type'}, inplace=True)
all_businesses.reset_index(drop=True, inplace=True)

In [195]:
def create_address(s):
    try:
        d = ast.literal_eval(s)
        return d
    except ValueError:
        return np.NaN

In [204]:
def create_dummy(s):
    d = {'delivery' :False, 'pickup': False}
    try:
        s = ast.literal_eval(s)
    except ValueError:
        return np.NaN
    if 'delivery' in s:
        d['delivery'] = True
    if 'pickup' in s:
        d['pickup'] = True
    return d

In [196]:
addresses = all_businesses.location.apply(create_address).apply(pd.Series).loc[:, ['zip_code', 'city', 'state']]

In [197]:
all_businesses = pd.concat([all_businesses, addresses], axis=1)

In [208]:
transactions = all_businesses.transactions.apply(create_dummy).apply(pd.Series).loc[:, ['delivery', 'pickup']]

In [210]:
all_businesses = pd.concat([all_businesses, transactions], axis=1)

In [213]:
all_businesses.drop(['location', 'transactions', 'url'], inplace=True, axis=1)

In [229]:
def convert_price(s):
    try:
        return len(s)
    except TypeError:
        return np.NaN

In [231]:
all_businesses.price = all_businesses.price.apply(convert_price)

In [243]:
#drop businesses not in Illinois
all_businesses = all_businesses[all_businesses['state'] == 'IL']

In [254]:
all_businesses

,id,alias,name,is_claimed,is_closed,review_count,rating,business_type,price,zip_code,city,state,delivery,pickup
0,2OGQfIsT33WC7sCKdqKuuw,carmelo-s-taco-place-chicago,Carmelo’s Taco Place,True,False,12.0,4.5,Restaurant,NaN,60629,Chicago,IL,True,True
1,YULX9j3fdcLtUNYHax19JA,maggie-gyros-and-chicken-chicago,Maggie Gyros & Chicken,False,False,5.0,2.5,Restaurant,NaN,60653,Chicago,IL,True,True
2,6hhWuqvuEIEescMsh807pw,morenas-kitchen-chicago,Morena's Kitchen,True,False,41.0,4.5,Restaurant,1.0,60639,Chicago,IL,True,True
3,VA1mmPgB4HcOpyw_-IA6DQ,northern-trust-chicago,Northern Trust,True,False,8.0,2.5,Restaurant,NaN,60603,Chicago,IL,False,False
4,YHToK3BYKGJudH_UNUCeVw,deli-and-more-chicago,Deli & More,True,False,9.0,4.5,Grocery Store,2.0,60657,Chicago,IL,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12447,Pv1Hx5hz9r_9KouNpq3kzg,restaurant-cuetzala-gro-chicago,Restaurant Cuetzala Gro,NaN,False,144.0,4.5,NaN,2.0,60626,Chicago,IL,True,True
12448,fGUGK_BQJGUYI2b3HAlFeA,tamalito-chicago,Tamalito,NaN,False,5.0,4.0,NaN,NaN,60626,Chicago,IL,True,True
12449,eNgSrwPnC1KrVmJL7CSptw,rogers-park-west-ridge-historical-society-chicago,Rogers Park West Ridge Historical Society,NaN,False,1.0,5.0,NaN,NaN,60626,Chicago,IL,False,False
12450,QHx-ZSbJ20FBW7QqEzCWhg,lighthouse-tavern-chicago,Lighthouse Tavern,NaN,False,47.0,4.5,NaN,1.0,60626,Chicago,IL,False,False


In [245]:
covid = pd.read_csv('COVID-19_Cases__Tests__and_Deaths_by_ZIP_Code.csv')

In [270]:
covid_small = covid[covid['Week Number'] == 53]
covid_small = covid_small.loc[:, ['ZIP Code', 'Cases - Cumulative', 'Tests - Cumulative', 'Deaths - Cumulative', 'Population']]

In [276]:
all_businesses = all_businesses.merge(covid_small, how='left', left_on='zip_code', right_on='ZIP Code').drop(['ZIP Code'], axis=1).rename(columns={'Cases - Cumulative': 'covid_cases', 'Tests - Cumulative': 'covid_tests', 'Deaths - Cumulative': 'covid_deaths', 'Population': 'zipcode_population'})

In [290]:
all_businesses.reset_index(drop=True, inplace=True)

In [297]:
all_businesses.to_csv('CSVs/all_businesses.csv')